In [14]:
ls Open3D-ML/ml3d/datasets/Paris/training_10_classes

IIITB_Base_train2_modified.ply*  part_a1_modified.ply*  reclassify_barriers.sh*
coarse_classes.xml*              part_a2_modified.ply*
col_drop.sh*                     part_a3_modified.ply*


In [2]:
import os

def filter_and_retype_ply(input_filename, output_filename, value_to_remove=12.0):
    """
    Reads an ASCII PLY file, removes points with a specific 
    'scalar_Classification' value, and changes the property type 
    in the header from float to int.
    """
    
    print(f"Reading from: {input_filename}")
    print(f"Writing to: {output_filename}")
    print(f"Removing points where scalar_Classification is {value_to_remove}...")
    
    if not os.path.exists(input_filename):
        print(f"Error: Input file not found at '{input_filename}'")
        return

    # Indices assumed from your header structure:
    # x: 0, y: 1, z: 2, scalar_Classification: 3, scalar_Original_cloud_index: 4
    CLASSIFICATION_INDEX = 3
    
    header_lines = []
    vertex_data_lines = []
    in_header = True
    original_vertex_count = 0
    
    try:
        # --- 1. Read and Process the Input File ---
        with open(input_filename, 'r') as infile:
            for line in infile:
                line = line.strip()
                if not line: # Skip empty lines
                    continue
                
                if in_header:
                    header_lines.append(line)
                    if line == 'end_header':
                        in_header = False
                else:
                    original_vertex_count += 1
                    parts = line.split()
                    
                    if len(parts) > CLASSIFICATION_INDEX:
                        try:
                            current_classification = float(parts[CLASSIFICATION_INDEX])
                            
                            # --- Filter Step: Keep only if classification is NOT the value to remove ---
                            if current_classification != value_to_remove:
                                # --- Retype Step: Change classification to int format for writing ---
                                parts[CLASSIFICATION_INDEX] = str(int(current_classification))
                                vertex_data_lines.append(' '.join(parts))
                                
                        except ValueError:
                            # If parsing fails, treat it as non-vertex data and skip filtering/retyping
                            vertex_data_lines.append(line)
                    else:
                        # Keep lines that aren't long enough to be standard vertex data
                        vertex_data_lines.append(line)


        # --- 2. Modify Header and Prepare Output ---
        modified_header = []
        new_vertex_count = len(vertex_data_lines)
        
        for line in header_lines:
            if line.startswith('element vertex'):
                # Update the vertex count
                modified_header.append(f'element vertex {new_vertex_count}')
            elif line.startswith('property float scalar_Classification'):
                # Change the type from float to int
                modified_header.append('property int scalar_Classification')
            else:
                # Keep other header lines as they are
                modified_header.append(line)

        # --- 3. Write the Output File ---
        with open(output_filename, 'w') as outfile:
            # Write the modified header
            outfile.write('\n'.join(modified_header) + '\n')
            
            # Write the filtered and retyped vertex data
            outfile.write('\n'.join(vertex_data_lines) + '\n')

        # --- 4. Summary ---
        points_removed = original_vertex_count - new_vertex_count
        
        print("-" * 45)
        print(f"✅ Processing complete!")
        print(f"Original points: {original_vertex_count}")
        print(f"Points removed (Classification=12.0): **{points_removed}**")
        print(f"Final points count: {new_vertex_count}")
        print(f"New file saved as: **{output_filename}**")
        print("Note: scalar_Classification property type is now 'int' in the header.")

    except Exception as e:
        print(f"An error occurred: {e}")

# --- CONFIGURATION ---
# IMPORTANT: Change 'input.ply' to the actual name of your file
INPUT_FILE = 'Manual/A1-A10_ongoing6.ply' 
OUTPUT_FILE = 'Open3D-ML/ml3d/datasets/Paris/training_10_classes/A1-A10_ongoing7.ply' 

# Run the function
filter_and_retype_ply(INPUT_FILE, OUTPUT_FILE, value_to_remove=12.0)

Reading from: Manual/A1-A10_ongoing6.ply
Writing to: Open3D-ML/ml3d/datasets/Paris/training_10_classes/A1-A10_ongoing7.ply
Removing points where scalar_Classification is 12.0...


KeyboardInterrupt: 

In [1]:
import open3d.ml as _ml3d
import open3d as o3d
import open3d.ml.torch as ml3d
import os
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import torch
print(torch.cuda.is_available())

True


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
device

device(type='cuda')

In [7]:
cfg_file = r"/mnt/e/PE/Mathew/Codes/Partitions/Open3D-ML/ml3d/configs/kpconv_parislille3d_mod.yml"
cfg = _ml3d.utils.Config.load_from_file(cfg_file)

model = ml3d.models.KPFCNN(**cfg.model)
cfg.dataset['dataset_path'] = r"/mnt/e/PE/Mathew/Codes/Partitions/Open3D-ML/ml3d/datasets/Paris"
dataset = ml3d.datasets.ParisLille3D(cfg.dataset.pop('dataset_path', None), **cfg.dataset)
pipeline = ml3d.pipelines.SemanticSegmentation(model, dataset=dataset, device="gpu", **cfg.pipeline)

In [8]:
ckpt_folder = "./logs/"
os.makedirs(ckpt_folder, exist_ok=True)
ckpt_path = ckpt_folder + "kpconv_parislille3d_202011241550utc_mod.pth"

In [9]:
# load the parameters.
# pipeline.load_ckpt(ckpt_path=ckpt_path)
pipeline.load_ckpt(ckpt_path=ckpt_path, is_resume=False)

In [10]:
dataset.get_split_list("train")

['/mnt/e/PE/Mathew/Codes/Partitions/Open3D-ML/ml3d/datasets/Paris/training_10_classes/part_a1_modified.ply',
 '/mnt/e/PE/Mathew/Codes/Partitions/Open3D-ML/ml3d/datasets/Paris/training_10_classes/part_a2_modified.ply',
 '/mnt/e/PE/Mathew/Codes/Partitions/Open3D-ML/ml3d/datasets/Paris/training_10_classes/part_a3_modified.ply']

In [11]:
train_split = dataset.get_split("train")
len(train_split)

3

In [12]:
%%time
pipeline.cfg.num_workers = 0
pipeline.cfg.pin_memory = False
pipeline.run_train()

validation: 100%|███████████████████████████████████████████████████████████████████████| 15/15 [02:17<00:00,  9.17s/it]


CPU times: user 1h 27min 58s, sys: 6min 12s, total: 1h 34min 10s
Wall time: 42min 38s
